### Решаем задачу автоматической проверки свободных ответов учеников на тестовые задания с помощью векторов и линейного классификатора
Пример тестового задания есть на картинке в репозитории, который клонируется во второй клетке с кодом. Датасет состоит из 3 колонок: id, текст ответа и оценка.

In [ ]:
# установка библиотеки tensorflow-text, USE использует ее для предобработки текстов

!pip install -U tensorflow-text

In [ ]:
# клонируем репозиторий с гитхаба

!git clone -l -s https://github.com/Ekaterina-Elmikeeva/Text-analytics.git
%cd Text-analytics/resources
!ls

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import tensorflow_text
from google.colab import files
import pandas as pd

from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as prfs

In [ ]:
# загружаем модель Universal Sentence Encoder

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
# считываем датасет, указываем название колонки с текстами и оценкой

data = pd.read_csv('data.csv', sep=';')
text_var = 'text'
class_var = 'mark'

In [ ]:
data.head() 

In [ ]:
# визуализация распреления оценок в датасете

df = data[class_var].value_counts(normalize=True) * 100
df.plot.bar(x=class_var)

In [ ]:
# оценка -1 ставится за отсутствие ответа. Исключаем из датасета ответы с такой оценкой

data = data[(data[class_var] != -1)]

In [ ]:
# в данном задании оценка ставится по трехбалльной шкале:
# 0 баллов за неправильный вариант, 1 балл за правильный вариант,
# 2 балла за правильный вариант и правильное объяснение своего ответа
# но попробуем сначала обучить модель отличать неправильные ответы (0 баллов)
# от сколько-нибудь правильных (1 и 2 балла), для этого все оценки 2
# превратим в 1, останется 2 класса и задача бинарной классификации

marks_new = []
for i, mark in enumerate(data[class_var]):
    if mark == 2:
        marks_new.append(1)
    else:
        marks_new.append(mark)
class_var = 'class_new'
data[class_var] = marks_new
del marks_new

In [ ]:
# устанавливаем размер обучающей выборки в процентах

train_vol = 0.7

In [ ]:
# делим датасет на обучающую и тестовую выборки
# из обучающей выборки удаляем дубликаты ответов

train, test = train_test_split(data, train_size = train_vol, random_state = 99, stratify = data[class_var])
train = train.drop_duplicates(subset=[text_var])

In [ ]:
# из тестовой выборки удаляем все ответы, которые есть в обучающей

check = []
train_data = train[text_var].tolist()
for text in test[text_var]:
    if text not in train_data:
        check.append('KEEP')
    else:
        check.append('IGNORE')
test['check'] = check
test = test.loc[test['check'] == 'KEEP']
test = test.drop(['check'], axis=1)
del check

In [ ]:
# переводим все ответы из обучающей выборки в вектора

train_embeddings = []
for text in train_data:
    embedding = embed(str(text))[0]
    train_embeddings.append(embedding)

In [ ]:
# переводим все ответы из тестовой выборки в вектора

test_embeddings = []
for text in test[text_var]:
    embedding = embed(str(text))[0]
    test_embeddings.append(embedding)

In [ ]:
# инициализируем классификаторы с параметрами по умолчанию

nbg = GaussianNB()
nbb = BernoulliNB()
lg = LogisticRegressionCV(cv=5, multi_class='ovr', max_iter=1200)
svm = SVC(kernel='poly', gamma='scale', probability=True)

In [ ]:
# получаем список оценок из обучающей и тестовой выборок

classes = train[class_var].tolist()
classes_to_check = test[class_var].tolist()

In [ ]:
# обучаем классификаторы

nbg.fit(train_embeddings,classes)
nbb.fit(train_embeddings,classes)
lg.fit(train_embeddings,classes)
svm.fit(train_embeddings,classes)

In [ ]:
# получаем оценки для ответов из тестовой выборки

classesNBG = nbg.predict(test_embeddings)
classesNBB = nbb.predict(test_embeddings)
classesLG = lg.predict(test_embeddings)
classesSVM = svm.predict(test_embeddings)

In [ ]:
# получаем точность, полноту и ф-меру для класса 0 (неправильные ответы)

metrics_NBG = prfs(classes_to_check, classesNBG, pos_label = 0, average = 'binary')
metrics_NBB = prfs(classes_to_check, classesNBB, pos_label = 0, average = 'binary')
metrics_LG = prfs(classes_to_check, classesLG, pos_label = 0, average = 'binary')
metrics_SVM = prfs(classes_to_check, classesSVM, pos_label = 0, average = 'binary')

In [ ]:
print(metrics_NBG,metrics_NBB,metrics_LG,metrics_SVM)

In [ ]:
# получаем точность, полноту и ф-меру для класса 1 (правильные ответы)

metrics_NBG = prfs(classes_to_check, classesNBG, pos_label = 1, average = 'binary')
metrics_NBB = prfs(classes_to_check, classesNBB, pos_label = 1, average = 'binary')
metrics_LG = prfs(classes_to_check, classesLG, pos_label = 1, average = 'binary')
metrics_SVM = prfs(classes_to_check, classesSVM, pos_label = 1, average = 'binary')

In [ ]:
print(metrics_NBG,metrics_NBB,metrics_LG,metrics_SVM)

### Код для проверки среднего результата на 10 прогонах

Когда мы один раз разделили датасет на обучающую и тестовую выборку, обучили классификатор и проверили качество, то эти метрики качества нельзя считать окончательными. Возможно, нам повезло, и обучающая выборка получилась хорошей. А если датасет разделить так, что на обучающей выборке классификатор обучится не так хорошо? Качество получится ниже.  

Чтобы получить адекватные метрики качества, нужно хотя бы 10 раз случайным образом, по-разному разделить датасет на обучающую и тестовую выборки и 10 раз обучить один и тот же классификатор. Затем вывести средние показатели качества. Код ниже проделывает эту операцию для 4 разных классификаторов и в конце выводит график для сравнения средних метрик.

In [ ]:
df_results = pd.DataFrame(columns=['classifier', 'train_size', 'average_precision',
                                   'average_recall', 'average_fscore'])
precision_all = {'NBG':[], 'NBB':[], 'SVM':[], 'LG':[]}
recall_all = {'NBG':[], 'NBB':[], 'SVM':[], 'LG':[]}
fscore_all = {'NBG':[], 'NBB':[], 'SVM':[], 'LG':[]}

data = pd.read_csv('data.csv', sep=';')

text_var = 'text'
class_var = 'mark'

data = data[data[class_var] != -1]

marks_new = []
for i, mark in enumerate(data[class_var]):
    if mark == 2:
        marks_new.append(1)
    else:
        marks_new.append(mark)
class_var = 'class_new'
data[class_var] = marks_new

train_vol = 0.7

for i in range(10):
    train, test = train_test_split(data, train_size = train_vol, stratify = data[class_var])
    train = train.drop_duplicates(subset=[text_var])
    check = []
    train_data = train[text_var].tolist()
    for text in test[text_var]:
        if text not in train_data:
            check.append('KEEP')
        else:
            check.append('IGNORE')
    test['check'] = check
    test = test.loc[test['check'] == 'KEEP']
    test = test.drop(['check'], axis=1)
    del check

    train_embeddings = []
    for text in train_data:
        embedding = embed(str(text))[0]
        train_embeddings.append(embedding)
    print('Прогон {}, вектора для обучения готовы.'.format(i+1))

    test_embeddings = []
    for text in test[text_var]:
        embedding = embed(str(text))[0]
        test_embeddings.append(embedding)
    print('Прогон {}, вектора для тестирования готовы.'.format(i+1))

    nbg = GaussianNB()
    nbb = BernoulliNB()
    svm = SVC(kernel='poly', gamma='scale', probability=True)
    lg = LogisticRegressionCV(cv=5, multi_class='ovr', max_iter=1200)

    classes = train[class_var].tolist()

    nbg.fit(train_embeddings,classes)
    nbb.fit(train_embeddings,classes)
    svm.fit(train_embeddings,classes)
    lg.fit(train_embeddings,classes)
    print('Прогон {}, все модели обучены.'.format(i+1))

    target = 0

    classesNBG = nbg.predict(test_embeddings)
    classesNBB = nbb.predict(test_embeddings)
    classesSVM = svm.predict(test_embeddings)
    classesLG = lg.predict(test_embeddings)

    classes_to_check = test[class_var].tolist()

    precisionNBG, recallNBG, fscoreNBG = prfs(classes_to_check, classesNBG, pos_label = 0, average = 'binary')[0:3]
    precisionNBB, recallNBB, fscoreNBB = prfs(classes_to_check, classesNBB, pos_label = 0, average = 'binary')[0:3]
    precisionSVM, recallSVM, fscoreSVM = prfs(classes_to_check, classesSVM, pos_label = 0, average = 'binary')[0:3]
    precisionLG, recallLG, fscoreLG = prfs(classes_to_check, classesLG, pos_label = 0, average = 'binary')[0:3]

    precision_all['NBG'].append(precisionNBG)
    precision_all['NBB'].append(precisionNBB)
    precision_all['SVM'].append(precisionSVM)
    precision_all['LG'].append(precisionLG)
    recall_all['NBG'].append(recallNBG)
    recall_all['NBB'].append(recallNBB)
    recall_all['SVM'].append(recallSVM)
    recall_all['LG'].append(recallLG)
    fscore_all['NBG'].append(fscoreNBG)
    fscore_all['NBB'].append(fscoreNBB)
    fscore_all['SVM'].append(fscoreSVM)
    fscore_all['LG'].append(fscoreLG)
    print('Прогон {} закончен.'.format(i+1))


for model in ['NBG', 'NBB', 'SVM', 'LG']:
    df_results = df_results.append({'classifier': model,
                                'average_precision':sum(precision_all[model])/10,
                                'average_recall':sum(recall_all[model])/10,
                                'average_fscore':sum(fscore_all[model])/10},
                                 ignore_index=True)

df_results.set_index('classifier')
df_results_t = df_results.transpose()
df_results_t.columns = df_results_t.iloc[0]
df_results_t.reset_index(inplace=True)
df_results_t = df_results_t.drop([0], axis=0)
df_results_t.plot.bar('index')

#### Ввести свой ответ на задание и проверить его на правильность

In [ ]:
your_answer=''

#необходимо перевести текст


#сделать прогноз для вашего ответа (метод predict)
